# Script Graph Networkdisk

In this notebook, I will provide a short presentation of my work. It led me to create a graph, stored on disk.

## Libraries

In [ ]:
import json
import sqlite3
import numpy as np
import networkdisk as nd
import time

## Conversion function (for lists)

In [ ]:
def convert_array(blob):
    return json.loads(blob)

## Creation of the graph

In [ ]:
def generate_graph(path = ""):

    start = time.process_time()

    sqlite3.register_converter('array', convert_array)
    con = sqlite3.connect(path + "articles_data.db", detect_types=sqlite3.PARSE_DECLTYPES)

    cu = con.cursor()
    qu = "SELECT Name_article, Links FROM Article"
    pe = list(cu.execute(qu))

    pe = [(node1, element) for node1, l in pe for element in l]
    length = len(pe)

    print("The data has been imported after {} seconds".format(time.process_time() - start))
    print("Its length is {}".format(length))
    
    G = nd.sqlite.DiGraph(db=path + "digraph.db", nom="A", insert_schema=True, sql_logger=True)

    ind = 80000000
    next_i = 0
    
    while next_i < length: # length of the whole db

        print("beginning to insert the next 80 000 000 edges")

        start = time.process_time()
        
        dat = pe[next_i:ind]

        print("data loaded")

        G.add_edges_from(dat)

        print("{} edges have been added".format(ind))
        print("after {} seconds".format(time.process_time() - start))
        print("\n")

        ind += 80000000
        next_i += 80000000
        dat = None

        time.sleep(1)

    G.helper.db.close()
    con.close()

In [ ]:
generate_graph("/data/name_user/")

I went over the database of articles by slices of 80.000.000 edges. It enabled me not to provide the whole set of edges (around 300 million) all at once to Networkdisk. 

The process took about 24h to complete. The result is a database, named *digraph.db*, in which are stored the nodes and edges.